BYNmasivofy2024*

In [775]:
# Importa las librerias a usar
import pandas as pd
import re
import numpy as np
import funcionesw as bynm
import datetime

importar arvhivos a procesar

In [776]:
# ubicacion de los archivos
patch1 = '../input/Kilometros en Servicio MIO - 05-Mar-2024.xlsx'# Se importa el archivo xlxs
patch2 =  '../input/REPORTE FINALIZADO 05_MARZ_2024.xlsx'
patch3 = '../input/JERARQUIAS.xlsx'

In [777]:
# Se importa el archivo xlxs
kmsmio = pd.read_excel(patch1, sheet_name='Programacion')
repofi = pd.read_excel(patch2, sheet_name='Programacion')
bitacora = pd.read_excel(patch2, sheet_name='Bitacora')
jerarquias = pd.read_excel(patch3, sheet_name='Hoja1')
# Se eliminan las filas y las columnas vacias
kmsmio = kmsmio.dropna(axis=0, how='all').dropna(axis=1, how='all')
repofi = repofi.dropna(axis=0, how='all').dropna(axis=1, how='all')
bitacora = bitacora.dropna(axis=0, how='all').dropna(axis=1, how='all')

In [778]:
# Se crea el df de adicionales
adi = kmsmio[kmsmio['Kilometros Programados Plan'].isna()]

Porcesar K2

In [779]:
# Filtra el DF por Kms en Servicio
kmsmio = kmsmio[kmsmio['Tipo Kilómetros']== 'Kms en Servicio']
kmsmio = kmsmio[kmsmio['Kilometros Programados Plan'].notna()]

In [780]:
def extraer_hora_regex(desde):
    """
    Esta funcion trata las columnas desde y hasta para verificar que las fechas esten iguales.
    Extrae la hora en formato hh:mm:ss de una cadena utilizando expresiones regulares.

    Parameters:
    desde (str): Cadena que contiene la hora en formato hh:mm:ss.

    Returns:
    str or None: La hora extraída en formato hh:mm:ss si se encuentra en la cadena, None si no se encuentra.
    """
    # Expresión regular para encontrar el patrón de hora (hh:mm:ss)
    patron_hora = r'(\d{2}):(\d{2}):(\d{2})'
    # Buscar el patrón en la cadena
    resultado = re.search(patron_hora, str(desde))
    # Si se encontró el patrón
    if resultado:
        # Extraer la hora, minutos y segundos encontrados
        hora = int(resultado.group(1))
        minutos = int(resultado.group(2))
        segundos = int(resultado.group(3))
        # Si la hora es 0, cambiarla a 24
        if hora == 0:
            hora = 24
        # Si la hora es 1, cambiarla a 25
        elif hora == 1:
            hora = 25
        # Devolver la hora modificada en formato hh:mm:ss
        return f"{hora:02d}:{minutos:02d}:{segundos:02d}"
    # Si no se encontró el patrón, devolver None
    return None


In [781]:
kmsmio['Desde'] = kmsmio['Desde'].apply(extraer_hora_regex)
kmsmio['hasta'] = kmsmio['hasta'].apply(extraer_hora_regex)

In [782]:
# Se crea una nueva columna 'id'
kmsmio['id'] = kmsmio.apply(lambda row: '-'.join((str(row['Fecha']), str(row['Línea']), str(row['Número de tarea vehículo']), 
                                                   str(row['Desde']), str(row['hasta']))), axis=1)


In [783]:
# Se verifica el tamaño del df
kmsmio.shape

(18645, 40)

In [784]:
# Se verifica que la columna id no tenga id repetidos
kmsmio['id'].value_counts().sort_values(ascending=False)

id
2024-03-05 00:00:00-A14A-1-05:00:00-05:12:10        1
2024-03-05 00:00:00-T47B-2-19:21:48-20:01:10        1
2024-03-05 00:00:00-A14A-1-05:26:00-05:38:10        1
2024-03-05 00:00:00-A14A-1-05:38:55-05:51:15        1
2024-03-05 00:00:00-A14A-1-05:52:00-06:04:43        1
                                                   ..
2024-03-05 00:00:00-A35A-99753-24:03:04-24:14:39    1
2024-03-05 00:00:00-A35A-99753-23:03:04-23:14:39    1
2024-03-05 00:00:00-A35A-99753-21:33:04-21:44:07    1
2024-03-05 00:00:00-A35A-99753-21:30:02-21:30:17    1
2024-03-05 00:00:00-A35A-99753-24:16:10-24:29:23    1
Name: count, Length: 18645, dtype: int64

Procesar reporte finalizado

In [785]:
repofi['desde'] = repofi['desde'].apply(extraer_hora_regex)
repofi['hasta'] = repofi['hasta'].apply(extraer_hora_regex)

In [786]:
# Filtra el DF por Kms en Servicio
repofi = repofi[repofi['Tipo de kilometros']== 'Kms en Servicio']
# Se crea una nueva columna 'id'
repofi['id'] = repofi.apply(lambda row: '-'.join((str(row['Fecha']), str(row['Línea']), str(row['Número de tarea vehículo']), 
                                                   str(row['desde']), str(row['hasta']))), axis=1)



In [787]:
# Se verifica que el id no tenga datos repetidos
repofi['id'].value_counts().sort_values(ascending=False)

id
2024-03-05 00:00:00-A14A-1-05:00:00-05:12:10        1
2024-03-05 00:00:00-T47B-2-19:21:48-20:01:10        1
2024-03-05 00:00:00-A14A-1-05:26:00-05:38:10        1
2024-03-05 00:00:00-A14A-1-05:38:55-05:51:15        1
2024-03-05 00:00:00-A14A-1-05:52:00-06:04:43        1
                                                   ..
2024-03-05 00:00:00-A35A-99753-24:03:04-24:14:39    1
2024-03-05 00:00:00-A35A-99753-23:03:04-23:14:39    1
2024-03-05 00:00:00-A35A-99753-21:33:04-21:44:07    1
2024-03-05 00:00:00-A35A-99753-21:30:02-21:30:17    1
2024-03-05 00:00:00-A35A-99753-24:16:10-24:29:23    1
Name: count, Length: 18645, dtype: int64

In [788]:
# Se verifica el tamaño del df
repofi.shape

(18645, 32)

In [789]:
# Se seleccionan las columnas del reporte finalizado
repofi = repofi[['id', 'Largo', 'Número de secuencia de carro', 'Número de tarea vehículo', 'Secuencia de arcos', 
                 'Número de viaje', 'Incumplidos','Descripción Novedad','Concesionario de Transporte', 'desde', 'hasta']]

In [790]:
# Prueba
repofi[repofi['Número de secuencia de carro']==53011]

,id,Largo,Número de secuencia de carro,Número de tarea vehículo,Secuencia de arcos,Número de viaje,Incumplidos,Descripción Novedad,Concesionario de Transporte,desde,hasta
6498,2024-03-05 00:00:00-A19A-222-06:24:05-06:53:19,7245,53011,222,A19A_Vuelta_18_29-10-2019_HABIL_SAB_DOM,16.0,NaN,NaN,BYNCPC,06:24:05,06:53:19
17553,2024-03-05 00:00:00-A13D-756-11:58:31-12:07:42,2803,53011,756,A13D_Ida_5_20-04-2023(HN),65.0,NaN,Caso: 2374666\nHora: 11:59\nLinea: A13D\nTarea...,BYNCPC,11:58:31,12:07:42
17554,2024-03-05 00:00:00-A13D-756-12:08:30-12:19:53,3269,53011,756,A13D_Vuelta_5_20-04-2023(HN),66.0,NaN,NaN,BYNCPC,12:08:30,12:19:53
17555,2024-03-05 00:00:00-A13D-756-12:20:31-12:29:42,2803,53011,756,A13D_Ida_5_20-04-2023(HN),67.0,NaN,NaN,BYNCPC,12:20:31,12:29:42
17556,2024-03-05 00:00:00-A13D-756-12:30:30-12:41:53,3269,53011,756,A13D_Vuelta_5_20-04-2023(HN),68.0,NaN,Caso: 2375138\nHora: 12:42\nLinea: A13D\nTarea...,BYNCPC,12:30:30,12:41:53


Se realiza el respectivo Merge con los dos df

In [791]:
# Se realiza el merge con outer en la columna id
merge = pd.merge(kmsmio, repofi, on='id', how='outer')

In [792]:
merge[merge.duplicated(subset='id')]

,Fecha,Día tipo,Designación de tarea vehículo,Línea,Tipo de viaje corto,Sentido,Tipo Kilómetros,Número de Vehículo,Concesionario de Transporte_x,Número de tarea vehículo_x,...,Largo_y,Número de secuencia de carro,Número de tarea vehículo_y,Secuencia de arcos_y,Número de viaje_y,Incumplidos_y,Descripción Novedad_y,Concesionario de Transporte_y,desde,hasta_y


In [793]:
# Prueba
merge[merge['Número de secuencia de carro']==53011]

,Fecha,Día tipo,Designación de tarea vehículo,Línea,Tipo de viaje corto,Sentido,Tipo Kilómetros,Número de Vehículo,Concesionario de Transporte_x,Número de tarea vehículo_x,...,Largo_y,Número de secuencia de carro,Número de tarea vehículo_y,Secuencia de arcos_y,Número de viaje_y,Incumplidos_y,Descripción Novedad_y,Concesionario de Transporte_y,desde,hasta_y
2333,2024-03-05,DHABIL,756,A13D,Viaje comercial,Ida,Kms en Servicio,NaN,NaN,756,...,2803,53011,756,A13D_Ida_5_20-04-2023(HN),65.0,NaN,Caso: 2374666\nHora: 11:59\nLinea: A13D\nTarea...,BYNCPC,11:58:31,12:07:42
2334,2024-03-05,DHABIL,756,A13D,Viaje comercial,Vuelta,Kms en Servicio,NaN,NaN,756,...,3269,53011,756,A13D_Vuelta_5_20-04-2023(HN),66.0,NaN,NaN,BYNCPC,12:08:30,12:19:53
2335,2024-03-05,DHABIL,756,A13D,Viaje comercial,Ida,Kms en Servicio,NaN,NaN,756,...,2803,53011,756,A13D_Ida_5_20-04-2023(HN),67.0,NaN,NaN,BYNCPC,12:20:31,12:29:42
2336,2024-03-05,DHABIL,756,A13D,Viaje comercial,Vuelta,Kms en Servicio,NaN,NaN,756,...,3269,53011,756,A13D_Vuelta_5_20-04-2023(HN),68.0,NaN,Caso: 2375138\nHora: 12:42\nLinea: A13D\nTarea...,BYNCPC,12:30:30,12:41:53
4056,2024-03-05,DHABIL,222,A19A,Viaje comercial,Vuelta,Kms en Servicio,53011,BYNCPC,222,...,7245,53011,222,A19A_Vuelta_18_29-10-2019_HABIL_SAB_DOM,16.0,NaN,NaN,BYNCPC,06:24:05,06:53:19


In [794]:
merge.columns

Index(['Fecha', 'Día tipo', 'Designación de tarea vehículo', 'Línea',
       'Tipo de viaje corto', 'Sentido', 'Tipo Kilómetros',
       'Número de Vehículo', 'Concesionario de Transporte_x',
       'Número de tarea vehículo_x', 'Desde', 'hasta_x', 'Duración',
       'Punto de inicio', 'Punto de término', 'Largo_x', 'Número de viaje_x',
       'Tipo de vehículo del viaje', 'Secuencia de arcos_x',
       'Descripción Novedad_x', 'Concesionario Programado',
       'Tipo de vehículo Programado', 'Incumplidos_x',
       'Kilometros Programados-Desvios', 'Kilometros Programados Plan',
       'Clase de viaje', 'a evaluar', 'TAREA', 'LINEA', 'VIAJE',
       'VEHICULO SGCO', 'Kms en Servicio (MTS PLANEADOS)', 'PARADAS PLANEADAS',
       'PARADAS REALIZADAS SGCO', '% PARADAS REALIZADAS SGCO',
       'Kms en Servicio (MTS SGCO)', 'FUENTE SGCO', 'HORA REAL SALIDA',
       'HORA REAL LLEGADA', 'id', 'Largo_y', 'Número de secuencia de carro',
       'Número de tarea vehículo_y', 'Secuencia de arcos

In [795]:
# Se verifica que el numero de filas en los df sea igual (True), para poder concluir que el merge fue exitoso
merge.shape[0] == kmsmio.shape[0] == repofi.shape[0]

True

Calificaciones de jerarquias en Bitacora

In [796]:
# Selecciona las columnas a usar
jerarquias = jerarquias[['JERARQUIA HGI RECO2',	'CALIFICACION']]

In [797]:
# Esto elimina los espacios en blanco adelante y atras
jerarquias['JERARQUIA HGI RECO2'] = jerarquias['JERARQUIA HGI RECO2'].str.strip()

In [798]:
#combina los el df de bitacora con el de jerarquias
bitacora = pd.merge(bitacora, jerarquias, left_on='Jerarquía', right_on='JERARQUIA HGI RECO2', how='left' )

In [799]:
# Esta linea es para verificar las filas que no se cruzaron por jerarquia y agregarlas a la matriz fe jererquia en input
x = bitacora[(bitacora['Jerarquía'].str.startswith('Salida'))& bitacora['CALIFICACION'].isnull()]
x['Jerarquía'].value_counts()

Series([], Name: count, dtype: int64)

In [800]:
# Este codigo elimina los duplicado de la columna calificaciones eliminando el que no afecta a kilometros

# Convertir la columna 'CALIFICACION' a numérica para poder comparar y encontrar el máximo
bitacora['CALIFICACION'] = pd.to_numeric(bitacora['CALIFICACION'], errors='coerce')

# Agrupar por las columnas 'Viaje' y 'Tarea', y mantener el valor máximo de 'CALIFICACION'
bitacora_grouped = bitacora.groupby(['Viaje', 'Tarea']).agg({'CALIFICACION': 'max'}).reset_index()

# Identificar las filas duplicadas en la tabla original
duplicados = bitacora[bitacora.duplicated(subset=['Viaje', 'Tarea'], keep=False)]

# Mantener solo una fila para cada grupo de duplicados en 'Viaje' y 'Tarea'
bitacora_sin_duplicados = bitacora.drop_duplicates(subset=['Viaje', 'Tarea'])

# Fusionar los datos filtrados con los valores máximos de 'CALIFICACION'
bitacora_final = pd.merge(bitacora_sin_duplicados, bitacora_grouped, on=['Viaje', 'Tarea'])

# Eliminar cualquiera de las filas duplicadas si ambas tienen valores nulos en 'CALIFICACION'
bitacora = bitacora_final.drop_duplicates(subset=['Viaje', 'Tarea'], keep='last', ignore_index=True)
    

In [801]:
# Se verifica que no hay duplicados
bitacora[bitacora.duplicated(subset=['Viaje', 'Tarea'], keep=False)]

,# Caso MC,Tipo novedad,Jerarquía,Tarea,Viaje,Parada,Operador,Línea,Vehículo,Hora,...,Jerarquia,Tipo de Novedad,Novedad,Retoma/Salida,Unnamed: 16,Unnamed: 17,Unnamed: 18,JERARQUIA HGI RECO2,CALIFICACION_x,CALIFICACION_y


In [802]:
# Se elimina la columna CALIFICACION_X y se renombra la columna CALIFICACION_Y
bitacora.drop(columns='CALIFICACION_x', axis=1, inplace=True)
bitacora.rename(columns={'CALIFICACION_y': 'CALIFICACION'}, inplace=True)


Merge con bitacora


In [803]:
# Se reemplan las filas nulas por ceros de las dos columnas para posteriormente carbiar el tipo de dato
bitacora[['Tarea', 'Viaje']] = bitacora[['Tarea', 'Viaje']].fillna(0).astype(int)

In [804]:
# Se selecionan las columnas de Bitacora que se van a usar
bitacora = bitacora[['Jerarquía',	'Tarea',	'Viaje', 'Parada', 'Vehículo', 'Novedad', 'CALIFICACION']]
# Se filtran los casos que no tiene numero de viaje ya que no tiene retoma
bitacora = bitacora[bitacora['Viaje'] != 0]

In [805]:
# Se compara las filas de los df, el resultado debe ser False
bitacora.shape[0]==bitacora.shape[0]

True

In [806]:
# Se agrega al merge anterior la informacion de bitacora
# Importante tener en cuenta los vehiculos que tengan mas de un caso en la bitacora, esto genera id duplicados.
mergebitacora = pd.merge(merge, bitacora, left_on=['Número de tarea vehículo_x', 'Número de viaje_x'], right_on=['Tarea', 'Viaje'], how='outer' )

In [807]:
# Revisar este resultado, ya que los valores que no coinceden deben ser iguales en el filtro
mergebitacora.shape[0]- merge.shape[0]

0

In [808]:
# Cambia Blanco y Negro Masivo por BYN en la columna Concesionario Programado
mergebitacora['Concesionario Programado'] = mergebitacora['Concesionario Programado'].replace('Blanco y Negro Masivo','BYN')

Revisar de numero de vehiculos para posteriormente verificar el incumplimiento

In [809]:
def vehiculos_xy_texto(row):
    """
    Asigna un texto descriptivo según las condiciones especificadas sobre los números de vehículo y secuencia de carro.

    Parámetros:
    row (pandas.Series): Una fila del DataFrame que contiene los datos de la bitácora.

    Retorna:
    str: Un texto descriptivo que indica el resultado de las comparaciones entre los números de vehículo y secuencia de carro.

    Detalles de la lógica:
    - Si el número de vehículo es nulo y el número de secuencia de carro no lo es, devuelve 'Revisar caso'.
    - Si el número de vehículo no es nulo y el número de secuencia de carro sí lo es, devuelve 'Se reemplaza por el del k2 por nan'.
    - Si el número de vehículo es diferente del número de secuencia de carro, devuelve 'Se reemplaza por el del k2 # diferente'.
    - En cualquier otro caso, devuelve 'Numero ok'.
    """
    if pd.isna(row['Número de Vehículo']) and pd.notna(row['Número de secuencia de carro']): 
        return 'Revisar caso # vehiculo'
    elif pd.notna(row['Número de Vehículo']) and pd.isna(row['Número de secuencia de carro']):
        return 'Se reemplaza por el del k2 por nan'
    elif row['Número de Vehículo'] != row['Número de secuencia de carro']:
        return 'Se reemplaza por el del k2 # diferente'
    else: 
        return 'Numero ok'

In [810]:
# Se crea una nueva columna donde se pondra los cambios que realizara la funcion
mergebitacora['texto_cmb_vehi'] = np.nan
# Se aplica la funcion al df
mergebitacora['texto_cmb_vehi'] = mergebitacora.apply(vehiculos_xy_texto, axis=1)

In [811]:
# Se verifica 
mergebitacora['texto_cmb_vehi'].value_counts()

texto_cmb_vehi
Se reemplaza por el del k2 por nan        9496
Numero ok                                 6445
Se reemplaza por el del k2 # diferente    2548
Revisar caso # vehiculo                    156
Name: count, dtype: int64

In [812]:
def vehiculo_xy(row):
    """
    Retorna el valor adecuado para el número de vehículo según las condiciones especificadas respecto al número de secuencia de carro.

    Parámetros:
    row (pandas.Series): Una fila del DataFrame que contiene los datos de la bitácora.

    Retorna:
    int or float or np.nan: El valor del número de vehículo seleccionado según las condiciones.

    Detalles de la lógica:
    - Si el número de vehículo es nulo y el número de secuencia de carro no lo es, devuelve el número de secuencia de carro.
    - Si el número de vehículo no es nulo y el número de secuencia de carro sí lo es, devuelve el número de vehículo.
    - Si el número de vehículo es diferente del número de secuencia de carro, devuelve el número de vehículo.
    - En cualquier otro caso, devuelve el mismo valor de número de secuencia de carro.
    """
    if pd.isna(row['Número de Vehículo']) and pd.notna(row['Número de secuencia de carro']): 
        return row['Número de secuencia de carro']
    elif pd.notna(row['Número de Vehículo']) and pd.isna(row['Número de secuencia de carro']):
        return row['Número de Vehículo']
    elif row['Número de Vehículo'] != row['Número de secuencia de carro']:
        return row['Número de Vehículo']
    else:
        return row['Número de secuencia de carro']

In [813]:
# Se aplica la funcion vehiculo_xy
mergebitacora['Número de secuencia de carro'] = mergebitacora.apply(vehiculo_xy, axis=1)

ok

Revision de casos amarillos y rosa

In [814]:
# Seleccion de casos Amarillos
amarillos = ['INGRESO::RETOMA','SALIDAS::POR FLOTA','SALIDAS::POR ACCIDENTE','SALIDAS::POR ABANDONO DE TAREA','INGRESO::ADICIONAL',
            'SALIDAS::POR VANDALISMO','SALIDAS::POR ALARMA','CANCELACION::DE ADICIONAL','SALIDAS::POR OPERADOR'
            ]



In [815]:
# Esta funcion marcara en una columna los cambios que se hagan a la columna Largo_y
def rev_amarillos_rosa_text(row):
    """
    Esta función se aplica a cada fila del DataFrame para evaluar si se cumplen ciertas condiciones
    basadas en las columnas 'Número de Vehículo', 'Descripción Novedad_x' y 'PARADAS PLANEADAS'.
    Según las condiciones cumplidas, devuelve un texto descriptivo correspondiente.

    Parámetros:
    row (pandas.Series): Una fila del DataFrame que contiene los datos de la bitácora.

    Retorna:
    str: Un texto descriptivo basado en las condiciones especificadas.

    Detalles de la lógica:
    - Si 'Número de Vehículo' no es NaN y 'Descripción Novedad_x' no es NaN, se evalúan las condiciones.
    - Si 'Descripción Novedad_x' contiene al menos uno de los elementos de la lista 'amarillos',
      se procede a realizar búsquedas específicas en 'Descripción Novedad_x'.
    - Se utilizan expresiones regulares para buscar patrones relacionados con 'INGRESO', 'SALIDAS' y 'CANCELACION'
      seguidos del número de parada en 'Descripción Novedad_x'.
    - Dependiendo de los resultados de las búsquedas y de los valores en 'PARADAS PLANEADAS', se devuelve un texto
      descriptivo que indica cómo se debe manejar el valor de 'Largo' en esa fila.

    """
    if pd.notna(row['Número de Vehículo']):
        if pd.notna(row['Descripción Novedad_x']): 
            # Casos rosa
            if isinstance(row['Descripción Novedad_x'], str) and 'Nro. Caso Desvio:' in row['Descripción Novedad_x']:
                    if row['Largo_y'] == row['Kilometros Programados Plan']:
                        return 'Se cambia Largo por el k2_if_8'
                    elif row['Largo_y'] != row['Kilometros Programados Plan']:
                        if row['Largo_y'] != row['Kilometros Programados-Desvios']:
                            if row['Largo_y'] < row['Largo_x']:
                                return 'Se cambia Largo por el k2__9'
                        else:
                            'Se mantiene largo_if_10'
            # Casos Amarillos                
            elif isinstance(amarillos, list) and any(isinstance(amarillo, str) and amarillo in row['Descripción Novedad_x'] for amarillo in amarillos):
                ingreso_match = re.search(r'INGRESO.*?Nro\. Parada:\s*(\d{1,3})', row['Descripción Novedad_x'])
                salida_match = re.search(r'SALIDAS.*?Nro\. Parada:\s*(\d{1,3})', row['Descripción Novedad_x'])
                cancelacion_match = re.search(r'CANCELACION.*?Nro\. Parada:\s*(\d{1,3})', row['Descripción Novedad_x'])       
                if ingreso_match and int(ingreso_match.group(1)) == 1 and int(row['PARADAS PLANEADAS']) != 1:
                    return 'Se cambia Largo por el k2_if1'
                elif ingreso_match and int(ingreso_match.group(1)) != 1:
                    return 'Se cambia Largo por el k2_if2'
                elif salida_match and int(salida_match.group(1)) != int(row['PARADAS PLANEADAS']):
                    return 'Se cambia Largo por el k2_if3'
                elif salida_match and int(salida_match.group(1)) == int(row['PARADAS PLANEADAS']):
                    return 'Se mantiene largo_if4'
                elif cancelacion_match and int(cancelacion_match.group(1)) != int(row['PARADAS PLANEADAS']):
                    return 'Se cambia Largo por el k2_if5'
                elif cancelacion_match and int(cancelacion_match.group(1)) == int(row['PARADAS PLANEADAS']):
                    return 'Se mantiene largo_if_6'
                else:
                    return 'Se mantiene largo_if7'                       
            else:
                return 'Se mantiene largo_if_11'
        else:
            return 'Se mantiene largo_if_12' 
    else:
        return 'Se mantiene largo_if_13' # Se agrega un caso adicional si el 'Número de Vehículo' es NaN


In [816]:
mergebitacora['texto_colores'] = np.nan

In [817]:
mergebitacora['texto_colores'] = mergebitacora.apply(rev_amarillos_rosa_text, axis=1)

In [818]:
mergebitacora['texto_colores'].value_counts()

texto_colores
Se mantiene largo_if_12           14954
Se mantiene largo_if_13            2506
Se cambia Largo por el k2_if_8      418
Se mantiene largo_if4               311
Se cambia Largo por el k2_if1       189
Se cambia Largo por el k2_if2       166
Se cambia Largo por el k2_if3        63
Se mantiene largo_if_11              30
Se mantiene largo_if_6                6
Se cambia Largo por el k2__9          1
Se cambia Largo por el k2_if5         1
Name: count, dtype: int64

In [819]:
# Esta funcion marcara en una columna los cambios que se hagan a la columna Largo_y
def rev_amarillos_rosa(row):
    """
    Aplica lógica condicional para marcar los cambios realizados en la columna 'Largo_y' en función de las características de la fila.

    Parámetros:
    row (pandas.Series): Una fila del DataFrame que contiene los datos de la bitácora.

    Retorna:
    int or float: El valor de 'Largo_y' después de aplicar las modificaciones, o el mismo valor si no se realizan cambios.

    Detalles de la lógica:
    - Verifica si el 'Número de Vehículo' no es NaN.
    - Verifica si la 'Descripción Novedad_x' no es NaN.
    - Si la 'Descripción Novedad_x' contiene 'Nro. Caso Desvio:', aplica la lógica específica para estos casos:
      - Si 'Largo_y' es igual a 'Kilometros Programados Plan', devuelve 'Kilometros Programados-Desvios'.
      - Si 'Largo_y' es diferente a 'Kilometros Programados Plan', pero igual a 'Kilometros Programados-Desvios', devuelve 'Largo_y'.
      - Si 'Largo_y' es menor que 'Largo_x', devuelve 'Largo_x', de lo contrario, devuelve 'Largo_y'.
    - Si la 'Descripción Novedad_x' contiene alguna de las cadenas definidas en la lista 'amarillos', aplica la lógica específica para estos casos:
      - Basado en patrones de expresiones regulares, determina si se deben realizar cambios en 'Largo_y'.
      - Si se cumplen las condiciones anteriores, devuelve 'Largo_x' o 'Largo_y' según corresponda.
    - Si ninguna de las condiciones anteriores se cumple, devuelve el valor actual de 'Largo_y'.

    Casos Especiales:
    - Si el 'Número de Vehículo' es NaN, devuelve el valor actual de 'Largo_y'.
    """
    # Lógica condicional para marcar los cambios en 'Largo_y' según las características de la fila
      
    if pd.notna(row['Número de Vehículo']):
        if pd.notna(row['Descripción Novedad_x']): 
            # Casos rosa
            if isinstance(row['Descripción Novedad_x'], str) and 'Nro. Caso Desvio:' in row['Descripción Novedad_x']:
                    if row['Largo_y'] == row['Kilometros Programados Plan']:
                        return row['Kilometros Programados-Desvios']
                    elif row['Largo_y'] != row['Kilometros Programados Plan']:
                        if row['Largo_y'] != row['Kilometros Programados-Desvios']:
                            if row['Largo_y'] < row['Largo_x']:
                                return row['Largo_x']
                            else:
                                row['Largo_y']
                        else:
                            row['Largo_y']
                    else:
                        row['Largo_y']
            # Casos Amarillos                
            elif isinstance(amarillos, list) and any(isinstance(amarillo, str) and amarillo in row['Descripción Novedad_x'] for amarillo in amarillos):
                ingreso_match = re.search(r'INGRESO.*?Nro\. Parada:\s*(\d{1,3})', row['Descripción Novedad_x'])
                salida_match = re.search(r'SALIDAS.*?Nro\. Parada:\s*(\d{1,3})', row['Descripción Novedad_x'])
                cancelacion_match = re.search(r'CANCELACION.*?Nro\. Parada:\s*(\d{1,3})', row['Descripción Novedad_x'])       
                if ingreso_match and int(ingreso_match.group(1)) == 1 and int(row['PARADAS PLANEADAS']) != 1:
                    return row['Largo_x']
                elif ingreso_match and int(ingreso_match.group(1)) != 1:
                    return row['Largo_x']
                elif salida_match and int(salida_match.group(1)) != int(row['PARADAS PLANEADAS']):
                    return row['Largo_x']
                elif salida_match and int(salida_match.group(1)) == int(row['PARADAS PLANEADAS']):
                    return  row['Largo_y']
                elif cancelacion_match and int(cancelacion_match.group(1)) != int(row['PARADAS PLANEADAS']):
                    return row['Largo_x']
                elif cancelacion_match and int(cancelacion_match.group(1)) == int(row['PARADAS PLANEADAS']):
                    return  row['Largo_y']
                else:
                    return  row['Largo_y']                       
            else:
                row['Largo_y']
        else:
            row['Largo_y'] 
    else:
        return row['Largo_y'] # Se agrega un caso adicional si el 'Número de Vehículo' es Na

In [820]:
mergebitacora['Largo_y'] = mergebitacora.apply(rev_amarillos_rosa, axis=1)

Casos verdes

comparar caso 53011

In [822]:
# Se seleccionan las columnas de adicional para el merge
adi = adi[['Número de Vehículo','Desde','hasta']]

In [823]:
adi[adi['Número de Vehículo']==53011]

,Número de Vehículo,Desde,hasta
20307,53011,11:59:31,12:08:42
20308,53011,12:09:30,12:20:53
20309,53011,12:21:31,12:30:42
20310,53011,12:31:30,12:42:53


In [824]:
# Convertir la columna 'Desde' a tipo datetime
adi['Desde'] = pd.to_datetime(adi['Desde'], format='%H:%M:%S')
adi['hasta'] = pd.to_datetime(adi['hasta'], format='%H:%M:%S')
# Restar un minuto a cada valor en la columna 'Desde'
adi['Desde'] = adi['Desde'] - pd.Timedelta(minutes=1)
adi['hasta'] = adi['hasta'] - pd.Timedelta(minutes=1)
adi['Desde'] = adi['Desde'].apply(extraer_hora_regex)
adi['hasta'] = adi['hasta'].apply(extraer_hora_regex)


In [825]:
# Crea una nueva columna llamada 'ID' que combine las columnas 'Número de secuencia de carro', 'Desde' y 'hasta_x'
mergebitacora['idadi'] = mergebitacora.apply(lambda row: f"{row['Número de secuencia de carro']}_{row['desde']}_{row['hasta_y']}", axis=1)

In [826]:
# Crea una nueva columna llamada 'ID' que combine las columnas 'Número de secuencia de carro', 'Desde' y 'hasta_x'
adi['idadi'] = adi.apply(lambda row: f"{row['Número de Vehículo']}_{row['Desde']}_{row['hasta']}", axis=1)

In [848]:
adi[adi.duplicated(subset='idadi', keep=False)]# Preguntar a cristan

,Número de Vehículo,Desde,hasta,idadi
20274,22117,16:10:09,16:39:28,22117_16:10:09_16:39:28
20275,22117,16:10:09,16:39:28,22117_16:10:09_16:39:28


In [828]:
x = mergebitacora[mergebitacora['Número de secuencia de carro']==53011]

In [829]:
def eliminar_no_realizadps(row):
    if pd.isna(row['Número de Vehículo']) and pd.isna(row['Número de secuencia de carro']):
        index = row.name
        mergebitacora.drop(index, axis=0, inplace=True) 

            

In [830]:
# Aplicar la función a cada fila del DataFrame mergebitacora
mergebitacora.apply(eliminar_no_realizadps, axis=1)


0        None
1        None
2        None
3        None
4        None
         ... 
18640    None
18641    None
18642    None
18643    None
18644    None
Length: 18645, dtype: object

In [847]:
mergebitacora[mergebitacora['Número de secuencia de carro']==53011]

,Fecha,Día tipo,Designación de tarea vehículo,Línea,Tipo de viaje corto,Sentido,Tipo Kilómetros,Número de Vehículo_x,Concesionario de Transporte_x,Número de tarea vehículo_x,...,Vehículo,Novedad,CALIFICACION,texto_cmb_vehi,texto_colores,idadi,Número de Vehículo_y,Desde_y,hasta,texto
5263,2024-03-05,DHABIL,222,A19A,Viaje comercial,Vuelta,Kms en Servicio,53011,BYNCPC,222,...,NaN,NaN,NaN,Numero ok,Se mantiene largo_if_12,53011_06:24:05_06:53:19,NaN,NaN,NaN,None
5264,2024-03-05,DHABIL,222,A19A,Viaje comercial,Ida,Kms en Servicio,53011,BYNCPC,222,...,23006.0,Ingreso adicional,NaN,Se reemplaza por el del k2 # diferente,Se mantiene largo_if_12,53011_06:54:37_07:28:31,NaN,NaN,NaN,None
5265,2024-03-05,DHABIL,222,A19A,Viaje comercial,Vuelta,Kms en Servicio,53011,BYNCPC,222,...,NaN,NaN,NaN,Se reemplaza por el del k2 # diferente,Se mantiene largo_if_12,53011_07:30:50_08:13:07,NaN,NaN,NaN,None
5266,2024-03-05,DHABIL,222,A19A,Viaje comercial,Ida,Kms en Servicio,53011,BYNCPC,222,...,NaN,NaN,NaN,Se reemplaza por el del k2 # diferente,Se mantiene largo_if_12,53011_08:13:37_08:41:43,NaN,NaN,NaN,None
5267,2024-03-05,DHABIL,222,A19A,Viaje comercial,Vuelta,Kms en Servicio,53011,BYNCPC,222,...,NaN,NaN,NaN,Se reemplaza por el del k2 # diferente,Se mantiene largo_if_12,53011_08:42:13_09:13:46,NaN,NaN,NaN,None
5268,2024-03-05,DHABIL,222,A19A,Viaje comercial,Vuelta,Kms en Servicio,53011,BYNCPC,222,...,NaN,NaN,NaN,Se reemplaza por el del k2 # diferente,Se mantiene largo_if_12,53011_09:40:11_10:08:46,NaN,NaN,NaN,None
5269,2024-03-05,DHABIL,222,A19A,Viaje comercial,Ida,Kms en Servicio,53011,BYNCPC,222,...,NaN,NaN,NaN,Se reemplaza por el del k2 # diferente,Se mantiene largo_if_12,53011_09:14:16_09:39:41,NaN,NaN,NaN,None
5270,2024-03-05,DHABIL,222,A19A,Viaje comercial,Vuelta,Kms en Servicio,53011,BYNCPC,222,...,NaN,NaN,NaN,Se reemplaza por el del k2 # diferente,Se mantiene largo_if_12,53011_10:36:35_11:04:02,NaN,NaN,NaN,None
5271,2024-03-05,DHABIL,222,A19A,Viaje comercial,Ida,Kms en Servicio,53011,BYNCPC,222,...,NaN,NaN,NaN,Se reemplaza por el del k2 # diferente,Se mantiene largo_if_12,53011_10:10:15_10:35:36,NaN,NaN,NaN,None
5272,2024-03-05,DHABIL,222,A19A,Viaje comercial,Vuelta,Kms en Servicio,53011,BYNCPC,222,...,53011.0,Cambio De Linea,2.0,Se reemplaza por el del k2 # diferente,Se mantiene largo_if4,53011_11:31:05_11:54:55,NaN,NaN,NaN,None


In [832]:

mergebitacora = pd.merge(mergebitacora, adi, on='idadi', how='left')

In [839]:
# Esta funcion en caso de tener adicioneales realiza el siguiente proceso
def verdes(row):
    if pd.notna(row['Desde_y']):
        row['Incumplidos_y'] = row['Incumplidos_x']
        row['Número de secuencia de carro'] = np.nan
        row['Largo_y'] = row['Largo_x']
        return 'ok'   

In [835]:
mergebitacora['texto'] = np.nan

In [853]:
mergebitacora['texto'] = mergebitacora.apply(verdes, axis=1)

In [850]:
x =mergebitacora[mergebitacora['texto'].notna()]

In [855]:
x[['Número de secuencia de carro','Número de Vehículo_x', 'Incumplidos_y','Incumplidos_x']]

,Número de secuencia de carro,Número de Vehículo_x,Incumplidos_y,Incumplidos_x
14120,53011,NaN,NaN,2.0
14121,53011,NaN,NaN,2.0
14122,53011,NaN,NaN,2.0
14123,53011,NaN,NaN,2.0
16089,22098,NaN,NaN,2.0
16090,22098,NaN,NaN,2.0
16091,22098,NaN,NaN,2.0
16092,22098,NaN,NaN,2.0


In [1268]:
# Elimina las columnas que no son necesarias
mergebitacora.drop(columns=['Número de Vehículo_y', 'Desde_y', 'hasta', 'texto'], inplace= True)

In [1269]:
mergebitacora.shape

(16139, 81)

In [1271]:
mergebitacora['Incumplidos_x'].value_counts()

Incumplidos_x
2.0    196
0.0     55
1.0     54
3.0      4
Name: count, dtype: int64

ok

Calificaciones incumplimineto 

In [1207]:
# Funcion replica de la funcion de Excel para el calculo de incumpliminetos, pero la siguiente funcion la anula, revisar si es necesaria
def col_calificaciones_funtion(row, df):
    """
    Calcula el valor del incumplimiento según las condiciones especificadas.

    Parámetros:
    row (pandas.Series): Una fila del DataFrame que contiene los datos de la bitácora.
    df (pandas.DataFrame): DataFrame que contiene los datos de la bitácora.

    Retorna:
    int or float or np.nan: El valor del incumplimiento calculado según las condiciones.
    """
    
    # Obtener el índice de la fila actual
    index = row.name
    
    # Calcular el índice de la fila siguiente
    next_index = index + 1
    next_index_next = index + 2
    
    # Obtener la fila siguiente si el índice es válido, de lo contrario, establecerla como None
    next_row = df.iloc[next_index] if next_index < len(df) else None
    next_row_next = df.iloc[next_index_next] if next_index_next < len(df) else None

    
    if row['Tipo Kilómetros'] == "Kms en Servicio" and pd.isna(row['Número de secuencia de carro']):
        if row['Tipo de viaje corto_x'] == "Viaje comercial":
            return 2
        elif row['Tipo de viaje corto_x'] in ["Viaje de punto operacional", "Viaje de posicionamiento"]:
            if next_row is not None and pd.isna(next_row['Número de secuencia de carro']):
                if (next_row['Incumplidos_y'] == 2) and (row['Largo_y'] in [0, 1]):
                    return 1
                elif (row['Largo_y'] not in [0, 1]) or (next_row['Largo_y']  in [0, 1]):
                    if pd.notna(next_row_next['Incumplidos_y']):
                        return next_row_next['Incumplidos_y'] 
                    else:
                        return 2
                else:
                    return next_row['Incumplidos_y']
            else:
                if row is not None and row['Largo_y'] in [0, 1]:
                    return 1
                else:
                    return 2
        else:
            return np.nan
    elif (row['Número de secuencia de carro'] in ["CAMBIO", "CANCELADA"]) and row['Tipo Kilómetros'] == "Kms en Vacio":
        return 0
    else:
        return np.nan


In [1208]:
#Aplicar la función a todo el DataFrame
mergebitacora['Incumplido_y'] = mergebitacora.apply(col_calificaciones_funtion, axis=1, args=(mergebitacora,))

In [1209]:
mergebitacora['Incumplidos_y'].value_counts()

Incumplidos_y
0.0    38
2.0    28
Name: count, dtype: int64

In [1210]:
def col_calificaciones(row, df):
    """
    Calcula el valor del incumplimiento según las condiciones especificadas.

    Parámetros:
    row (pandas.Series): Una fila del DataFrame que contiene los datos de la bitácora.
    df (pandas.DataFrame): DataFrame que contiene los datos de la bitácora.

    Retorna:
    int or float or np.nan: El valor del incumplimiento calculado según las condiciones.
    """
   
    if pd.isna(row['Incumplidos_x']) and pd.notna(row['Incumplidos_y']):
        return np.nan
    elif row['Incumplidos_x'] == 0:
        return 0
    elif row['Incumplidos_x'] in [2,3] and row['CALIFICACION'] == 1:
        return row['CALIFICACION']
    elif row['Incumplidos_x'] == 3:
        return row['Incumplidos_x']
    elif row['Incumplidos_x'] in [0,1,3] and row['Incumplidos_y']== 2:
        return row['Incumplidos_x'] 
    elif row['Incumplidos_x']== 2 and  row['Incumplidos_y']== 2:
        return row['Incumplidos_x'] 
    else:
        np.nan


In [1214]:
# Aplicar la función a todo el DataFrame
mergebitacora['Incumplidos_y'] = mergebitacora.apply(col_calificaciones, axis=1, args=(mergebitacora,))

In [1215]:
mergebitacora['Incumplidos_y'].value_counts()

Incumplidos_y
0.0    55
2.0    11
1.0     4
3.0     2
Name: count, dtype: int64

In [1152]:
mergebitacora.shape

(16139, 81)

OK pendiente revisar calificaciones

In [45]:
z = mergebitacora[(mergebitacora['Incumplidos_x'].isna())&(mergebitacora['Incumplidos_y'].notna())]

In [46]:
z[['VIAJE', 'TAREA', 'Incumplidos_x', 'Incumplidos_y']]

,VIAJE,TAREA,Incumplidos_x,Incumplidos_y


In [47]:
mergebitacora[['VIAJE', 'TAREA', 'Incumplidos_x', 'Incumplidos_y']]

,VIAJE,TAREA,Incumplidos_x,Incumplidos_y
0,1.0,1.0,NaN,NaN
1,2.0,1.0,NaN,NaN
2,9.0,1.0,NaN,NaN
3,10.0,1.0,NaN,NaN
4,17.0,1.0,NaN,NaN
...,...,...,...,...
18640,9955.0,99753.0,NaN,NaN
18641,9956.0,99753.0,NaN,NaN
18642,9957.0,99753.0,NaN,NaN
18643,9958.0,99753.0,NaN,NaN


verificacion

In [48]:
##
##
##no va
##
##
# Definir una función para aplicar a cada grupo
def eliminar_duplicados_bitacora(row):
    # Verificar si hay al menos un valor en 'Incumplidos_x'
    if row['Incumplidos_x'].isnull() and row['Incumplidos_x'].isnull():   
        return row[row['Incumplidos_x'].notnull()]  # Conservar la fila con 'Incumplidos_x' no nulo
    # Verificar si hay al menos un valor en 'Incumplidos_y'
    elif row['Incumplidos_y'].notnull().any():
        return row[row['Incumplidos_y'].notnull()]  # Conservar la fila con 'Incumplidos_y' no nulo
    # Conservar la fila con el valor más bajo en 'CALIFICACION'
    else:
        return row.loc[row['CALIFICACION'].idxmin()]

In [49]:
# Muestra los id duplicados producto del merge con bitacora
mergebitacora[mergebitacora['id'].duplicated()]

,Fecha_x,Día tipo_x,Designación de tarea vehículo_x,Línea_x,Tipo de viaje corto_x,Sentido_x,Tipo Kilómetros,Número de Vehículo,Concesionario de Transporte_x,Número de tarea vehículo_x,...,Kms cancelados,Jerarquía,Tarea,Viaje,Parada,Vehículo_y,Novedad,CALIFICACION,texto_cmb_vehi,Incumplido_y


Revision de Adicionales

Columna condicional 

In [50]:
# Se cambian los vehiculos del cierre por los de k2##
mergebitacora['Número de secuencia de carro'] = [x if pd.notna(x) else y for x, y in zip(mergebitacora['Número de secuencia de carro'], mergebitacora['Número de Vehículo'])]
# Marca en la columnas Cambios los cambios que realiza el código
mergebitacora['condicion'] = [ np.nan if  pd.notna(x) else 'Se Cambia # V RF' for x, y in zip(mergebitacora['Número de secuencia de carro'], mergebitacora['Número de Vehículo'])]


In [51]:
# # Función para actualizar el texto de la columna condicion para Largo_y si Largo_x es mayor
def actualizar_largo_texto(row):
    if pd.isna(row['Largo_y']) and pd.isna(row['condicion']) :
        return 'Cambia largo por NAN'
    elif pd.isna(row['Largo_y']) and pd.notnull(row['condicion']) :
        return row['condicion'] + '/Cambia largo por NAN'
    elif row['Largo_x'] > row['Largo_y'] and pd.isna(row['condicion']):
        return 'Cambia x Mayor largo'
    elif row['Largo_x'] > row['Largo_y'] and pd.notnull(row['condicion']) :
        return row['condicion'] + '/Cambia x Mayor largo'
    else:
        if pd.notnull(row['condicion']):
            return row['condicion']
            

In [52]:
# Función para actualizar los valores de Largo_y si Largo_x es mayor
def actualizar_largo(row):
    if pd.isna(row['Largo_y']):
        return row['Largo_x']
    elif row['Largo_x'] > row['Largo_y']:
        return row['Largo_x']
    else:
        return row['Largo_y']

In [53]:
# Marcar las filas donde se realizan cambios en una nueva columna
mergebitacora['condicion'] = mergebitacora.apply(actualizar_largo_texto, axis=1)

In [54]:
# Aplicar la función a la columna Largo_y
mergebitacora['Largo_y'] = mergebitacora.apply(actualizar_largo, axis=1)

In [55]:
#Se verifican los cambios realizados
mergebitacora['condicion'].value_counts()

condicion
Cambia largo por NAN                     15876
Se Cambia # V RF/Cambia largo por NAN     2350
Name: count, dtype: int64

In [56]:
'Número de secuencia de carro', 'Secuencia de arcos', 'Concesionario Programado'

('Número de secuencia de carro',
 'Secuencia de arcos',
 'Concesionario Programado')

In [57]:
# Condiciones para la columna condicional
condicion1 = mergebitacora['Largo_x'] != mergebitacora['Largo_y']
condicion2 = mergebitacora['Número de Vehículo'] != mergebitacora['Número de secuencia de carro']
condicion3 = mergebitacora['Secuencia de arcos_x'] != mergebitacora['Secuencia de arcos_y']
condicion4 = mergebitacora['Concesionario de Transporte_x'] != mergebitacora['Concesionario Programado']

# Aplicar condiciones y asignar valores a la nueva columna
mergebitacora['condicional2'] = np.where(condicion1 & condicion2 & condicion3 & condicion4, 'Revisar largo, parada, vehículo y consecionario',
                                    np.where(condicion1 & condicion2 & condicion3, 'Revisar largo, parada y vehículo',
                                        np.where(condicion1 & condicion2 & condicion4, 'Revisar largo, parada y consecionario',         
                                            np.where(condicion1 & condicion3 & condicion4, 'Revisar largo, parada y consecionarios',
                                                np.where(condicion2 & condicion3 & condicion4, 'Revisar vehiculo, parada y consecionarios',        
                                                    np.where(condicion1 & condicion2, 'Revisar largo y vehiculo',
                                                        np.where(condicion1 & condicion3, 'Revisar largo y parada',
                                                            np.where(condicion1 & condicion4, 'Revisar largo y consecionario', 
                                                                np.where(condicion2 & condicion3, 'Revisar vehiculo y parada',
                                                                    np.where(condicion2 & condicion4, 'Revisar vehiculo y consecionario',        
                                                                        np.where(condicion3 & condicion4, 'Revisar parada y consecionario',                                            
                                                                            np.where(condicion1, 'Revisar largo', 
                                                                                np.where(condicion2, 'Revisar numero de vehiculo',
                                                                                    np.where(condicion3, 'Revisar paradas',
                                                                                        np.where(condicion4, 'Revisar consesionario', 'OK')))))))))))))))

In [58]:
mergebitacora['condicional2'].value_counts()

condicional2
OK                                  10235
Revisar consesionario                5875
Revisar vehiculo y consecionario     2506
Revisar largo                          28
Revisar largo y consecionario           1
Name: count, dtype: int64

In [59]:
# Exporta a un archivo excel el Df
#mergebitacora.to_excel('../output/prueba_mergebita.xlsx', index=False, sheet_name='k2_procesado')